In [1]:
import logging
logging.basicConfig(level=logging.INFO)

## __get__(self, instance, owner)

instance 는 디스크립터를 호출한 인스턴스를 의미한다. (아래 예제에서 client 인스턴스)

owner 는 디스크립터를 호출한 인스턴스의 Class를 의미한다. (아래 예제에서 ClientClass 의미)

In [2]:
class DescriptorClass:
    def __get__(self, instance, owner):
        if instance is None:
            return f"{self.__class__.__name__}.{owner.__name__}"
        return f"value for {instance}"
    
class ClientClass:
    descriptor = DescriptorClass()

ClientClass.descriptor  #<-- Class

'DescriptorClass.ClientClass'

In [7]:
ClientClass().descriptor  #<-- 괄호가 있으면 Instance를 의미한다.
print(ClientClass().descriptor)
print(ClientClass().__dict__)

value for <__main__.ClientClass object at 0x0000009A181D2850>
{}


In [5]:
client = ClientClass()
print (client.descriptor)
print (client.__dict__)

value for <__main__.ClientClass object at 0x0000009A181D2490>
{}


In [8]:
class DescriptorClass:
    def __get__(self, instance, owner):
        if instance is None:
            return self
        
        logging.info("Call: %s.__get__(%r, %r)", self.__class__.__name__, instance, owner)
        print ('1. 인스턴스의 __dict__')
        print (instance.__dict__)
        print ('2. 인스턴스 클래스의 __dict__')
        print (owner.__dict__)
        return instance

class ClientClass:
    descriptor = DescriptorClass()

In [9]:
print ('min')
client = ClientClass()
print ('--')
client.descriptor

INFO:root:Call: DescriptorClass.__get__(<__main__.ClientClass object at 0x0000009A181D2820>, <class '__main__.ClientClass'>)


min
--
1. 인스턴스의 __dict__
{}
2. 인스턴스 클래스의 __dict__
{'__module__': '__main__', 'descriptor': <__main__.DescriptorClass object at 0x0000009A175BF130>, '__dict__': <attribute '__dict__' of 'ClientClass' objects>, '__weakref__': <attribute '__weakref__' of 'ClientClass' objects>, '__doc__': None}


In [12]:
print (client.__dict__)    # 새로 만든 인스턴스여서 __dict__가 비어있음

{}


In [11]:
print (ClientClass.__dict__)

{'__module__': '__main__', 'descriptor': <__main__.DescriptorClass object at 0x0000009A175BF130>, '__dict__': <attribute '__dict__' of 'ClientClass' objects>, '__weakref__': <attribute '__weakref__' of 'ClientClass' objects>, '__doc__': None}


## __set__(self, instance, value)

descriptor = Field(...)

Field 가 descriptor 인스턴스 생성할때 __init__ 호출되고 다음에 __set_name__  호출되면서 self._name 을 선언한다.

한번 호출되면 self._name 은 계속 유지된다.

In [62]:
class Validation:
    def __init__(self, validation_function, error_msg:str):
        print('1')
        self.validation_function = validation_function   # lambda 함수 저장
        self.error_msg = error_msg                       # error 메세지 저장
    def __call__(self, value):
        if not self.validation_function(value):
            raise ValueError(f"{value!r} {self.error_msg}")

class Field:
    def __init__(self, *validations):
        print('2')
        self._name = None
        self.validations = validations   # validations 에 Validation 객체를 저장한다.
    def __set_name__(self, owner, name):
        print('3')
        self._name = name
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__[self._name]
    def validate(self, value):
        for validation in self.validations:
            validation(value)     # Validation Instance의 __call__ 함수 호출
    def __set__(self, instance, value):
        self.validate(value)
        instance.__dict__[self._name] = value

class ClientClass:
    descriptor = Field(
        Validation(lambda x: isinstance(x, (int, float)), "는 숫자가 아님"),
        Validation(lambda x: x >=0, "는 0보다 작음"),        
    )

1
1
2
3


In [58]:
client = ClientClass()

In [59]:
client.descriptor = 42

In [60]:
client.descriptor

42

In [61]:
client.descriptor = -42

ValueError: -42 는 0보다 작음

In [18]:
client.descriptor = "invalid value"

ValueError: 'invalid value' 는 숫자가 아님

## __set_name__(self, owner, name)

owner : 디스크립터를 호출한 인스턴스의 Class를 의미한다. (아래 예제에서 ClientClass 의미)

name : 디스크립터의 인스턴스 이름 (아래 예제에서 descriptor)

디스크립터를 인스턴스 선언할때 디스크립터의 __init__ 호출 된후 __set_name__ 이 호출 된다.

아래 예제에서는 __set_name__(self, owner, name) 의 name 이 즉 인스턴스 이름을 속성 이름으로 선언했다.

일반적으로 클래스에 디스크립터 객체를 만들 때는 디스크립터가 처리하려는 속성의 이름을 알아야 한다.

속성의 이름은 __dict__ 에서 __get__ 과 __set__ 메서드로 읽고 쓸 때 사용한다.

파이썬 3.6 이전에는 디스크립터가 이 이름을 자동으로 설정하지 못했기 때문에 보통은 객체 초기화 시 명시적으로 이름을 전달했다.

이렇게 해도 잘 동작하지만 새로운 속성에 대한 디스크립터를 추가할 때마다 이름을 복사해야 하는 불편함이 있었다.


#### __set_name__ 이 없을 때의 전형적인 디스크립터 코드

In [17]:
class DescriptorWithName:
    def __init__(self, name):
        self.name = name
    def __get__(self, instance, value):
        if instance is None:
            return self
        logging.info("%r에서 %r 속성 가져오기", instance, self.name)
        return instance.__dict__[self.name]
    def __set__(self, instance, value):
        instance.__dict__[self.name] = value

class ClientClass:
    descriptor = DescriptorWithName("descriptor")

In [18]:
client = ClientClass()
client.descriptor = "value"
client.descriptor

INFO:root:<__main__.ClientClass object at 0x0000009A175BFA60>에서 'descriptor' 속성 가져오기


'value'

In [20]:
client = ClientClass()
print(client.__dict__)


{}


#### __set_name__을 사용한 디스크립터 코드

In [65]:
class DescriptorWithName:
    def __init__(self, name=None):
        self.name = name                   # 초기화시 이름을 선언을 안하면 None 으로 선언된다.
    def __set_name__(self, owner, name):
        print ('[log] __set_name__')
        print ('      - owner = ', owner)
        print ('      - name  = ', name)
        self.name = name
    def __get__(self, instance, value):
        if instance is None:
            return self
        logging.info("%r에서 %r 속성 가져오기", instance, self.name)
        return instance.__dict__[self.name]
    def __set__(self, instance, value):
        instance.__dict__[self.name] = value
class ClientClass:
    print ('[log] 1')
    descriptor = DescriptorWithName()
    imsi = DescriptorWithName('test')

[log] 1
[log] __set_name__
      - owner =  <class '__main__.ClientClass'>
      - name  =  descriptor
[log] __set_name__
      - owner =  <class '__main__.ClientClass'>
      - name  =  imsi


In [28]:
client = ClientClass()
client.descriptor = "value"
client.descriptor

INFO:root:<__main__.ClientClass object at 0x0000009A175BF370>에서 'descriptor' 속성 가져오기


'value'

## 이상적인 구현방법

만약 실질적인 코드 반복의 증거가 없거나 복잡성의 대가가 명확하지 않다면 굳이 디스크립터를 사용할 필요가 없다.

[1] 속성의 이름은 디스크립터에 할당된 변수 중 하나로 여기서는 current_city이다. 그리고 이에 대한 추적을 저장할 변수의 이름을 디스크립터에 전달한다. 이 예에서는 cities_visited 라는 속성에 current_city의 모든 값을 추적하도록 지시한다.

[2] 디스크립터를 처음으로 호출할 때는 추적 값이 존재하지 않을 것이므로 나중에 추가할 수 있도록 비어 있는 배열로 초기화 한다.

[3] 처음 Traveller를 호출할 때는 방문자가 없으므로 인스턴스 사전에서 current_city의 키도 존재하지 않을 것이다. 이런 경우도 새로운 여행지가 생긴 것이므로 추적의 대상이 된다. 앞에서 목록을 초기화 하는 것과 비슷한 이유이다.

[4] 새 값이 현재 설정된 값과 다른 경우에만 변경 사항을 추적한다.

[5] Traveller의 __init__ 메서드에서 디스크립터가 이미 생성된 단계이다. 할당 명령은 2단계 값을 추적하기 위한 빈 리스트 만들기를 실행하고, 3단계를 실행하여 리스트에 값을 추가하고 나중에 검색하기 위한 키를 설정한다.

[6] 사전의 setdefault 메서드는 KeyError를 피하기 위해 사용된다. setdefault는 두 개의 파라미터를 받는데 첫 번째 파라미터의 키가 있으면 해당 값을 반환하고 없으면 두 번째 파라미터를 반환한다.

In [3]:
class HistoryTraceAttribute:
    def __init__(self, trace_attribute_name) -> None:
        self.trace_attribute_name = trace_attribute_name #[1]
        self._name = None
    def __set_name__(self, owner, name):
        self._name = name
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__[self._name]
    def __set__(self, instance, value):
        self._track_change_in_value_for_instance(instance, value)
        instance.__dict__[self._name] = value
    def _track_change_in_value_for_instance(self, instance, value):
        self._set_default(instance) # [2]
        if self._needs_to_track_change(instance, value):
            instance.__dict__[self.trace_attribute_name].append(value)
    def _needs_to_track_change(self, instance, value) -> bool:
        try:
            current_value = instance.__dict__[self._name]
        except KeyError: # [3]
            return True
        return value != current_value # [4]
    def _set_default(self, instance):
        instance.__dict__.setdefault(self.trace_attribute_name, []) # [6]

class Traveller:
    current_city = HistoryTraceAttribute("cities_visited") # [1]
    def __init__(self, name, current_city):
        self.name = name
        self.current_city = current_city # [5]

In [12]:
alice = Traveller('Alice', 'Barcelona')

In [13]:
alice.travel = "Paris"
alice.travel = "Brussels"
alice.travel = "Amsterdam"

In [6]:
p



In [14]:
print(alice.current_city)

Barcelona
